In [1]:
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt

# Generamos nuestro dataset

# data = [0.23,-0.88,-0.34,.12,.13,.56,-.5,-.4,.12,0.23,0.98,-0.12,-0.05,.91,.99,.78,.87,-.03,.89,0.97]
# data2 =  [-1,-1,-1,-1,-1,1,-1,-1,-1,-1,1,-1,-1,1,1,1,1,-1,1,1]
data = [[0.23,-0.88,-0.34,0.12,0.13,0.56,-0.5,-0.4,0.12,0.23,0.98,-0.12,-0.05,0.91,0.99,0.78,0.87,-0.03,0.89,0.97],
        [-1,-1,-1,-1,-1,1,-1,-1,-1,-1,1,-1,-1,1,1,1,1,-1,1,1]]
print(data)
labels =  [-1,-1,-1,-1,-1,1,-1,-1,-1,-1,1,-1,-1,1,1,1,1,-1,1,1]


#plt.scatter(data, np.zeros(len(data)), color = "blue")
#plt.scatter(data[:len(data)//4], np.zeros(len(data)//4), color = "orange")
#plt.scatter(data[-len(data)//4:], np.zeros(len(data)//4), color = "orange")

[[0.23, -0.88, -0.34, 0.12, 0.13, 0.56, -0.5, -0.4, 0.12, 0.23, 0.98, -0.12, -0.05, 0.91, 0.99, 0.78, 0.87, -0.03, 0.89, 0.97], [-1, -1, -1, -1, -1, 1, -1, -1, -1, -1, 1, -1, -1, 1, 1, 1, 1, -1, 1, 1]]


## Cargar los datos

In [ ]:
data = np.genfromtxt("challenge_train.csv", delimiter=",", skip_header=1)
datos = data[:,1:-1]
# labels = data[:,-1:]
print(datos.shape)
# print(labels.shape)

print(len(datos))
print(len(datos[0]))

In [17]:
# Entrenamiento estandar de una red neuronal cuántica

def feature_map(x, param, i):
    print(i)
    qml.Hadamard(wires = i)
    qml.RZ(x*param , wires = i)

def variational_block(params):
    for i in range(len(params)-1):
        for j in range(i+1,len(params)):
            print(i,"-",j)
            qml.CNOT(wires = [i,j])
    


dev = qml.device("default.qubit", wires = 2)
@qml.qnode(dev)
def circuit(x, params):
    print(len(params)-1,"x:",x)
    for dato in x:
        print(dato)
        # feature_map(dato,params[i],i)
    # variational_block(params)
    # return qml.expval(qml.PauliZ(len(params)-1))
    return qml.expval(qml.Hermitian(x, wires = [0,1]))


def error(qnode, data, labels, params):
    err = 0
    for i,fila in enumerate(data):
        print(i, "-", fila)
        print(params)
        d = qnode(fila, params)
        err -= (labels[i] * d[0])
    return err / len(data[0])

def accuracy(qnode,data, labels, params):
    n = 0
    for i,fila in enumerate(data):
        if labels[i] == np.sign(qnode(fila, params[i])[0]):
            n += 1
        if labels[i] == np.sign(qnode(fila, params[i])[1]):
            n += 1
    return n / (len(data[0])*2)


def train(data, labels, error_function, params, stepsize = 0.1, epochs = 100, plot = True):
    print(len(data))
    dev = qml.device("default.qubit", wires = len(data))
    qnode = qml.QNode(circuit, dev)
    
    opt = qml.AdagradOptimizer(stepsize = stepsize)

    costs = []
    for epoch in range(epochs):
        params, cost = opt.step_and_cost(lambda params: error_function(qnode,data, labels, params), params)
        costs.append(cost)

    if plot:
        plt.plot(costs)

    return params

In [18]:
initial_params = np.random.rand(len(data)-1, requires_grad = True)
print(initial_params)
params = train(data, labels, error, initial_params)
print(accuracy(data, labels, params))

[0.53293653]
2
0 - [0.23, -0.88, -0.34, 0.12, 0.13, 0.56, -0.5, -0.4, 0.12, 0.23, 0.98, -0.12, -0.05, 0.91, 0.99, 0.78, 0.87, -0.03, 0.89, 0.97]
Autograd ArrayBox with value [0.53293653]
0 x: [0.23, -0.88, -0.34, 0.12, 0.13, 0.56, -0.5, -0.4, 0.12, 0.23, 0.98, -0.12, -0.05, 0.91, 0.99, 0.78, 0.87, -0.03, 0.89, 0.97]
0.23
-0.88
-0.34
0.12
0.13
0.56
-0.5
-0.4
0.12
0.23
0.98
-0.12
-0.05
0.91
0.99
0.78
0.87
-0.03
0.89
0.97


ValueError: Observable must be a square matrix.

In [ ]:
def feature_map(x, feature_param):
    qml.Hadamard(wires = 0)
    qml.RZ(x * feature_param, wires = 0)

def feature_error(data, labels, feature_param):

    dev = qml.device("default.qubit", wires = 1)
    @qml.qnode(dev)
    def circuit(x1, x2):
        feature_map(x1, feature_param)
        qml.adjoint(feature_map)(x2, feature_param)
        return qml.probs(wires = 0)

    err = 0
    for i in range(len(data)):
        for j in range(i, len(data)):
            d = circuit(data[i], data[j])
            err -= labels[i] * labels[j] * d[0][0]
    return err


In [ ]:
initial_param = np.random.rand(1, requires_grad = True)
param = train(data, labels, feature_error, initial_param)

In [ ]:
print(accuracy(data, labels, params))